### Importing libraries

In [ ]:
import sys
import numpy as np
import nltk
import os
import re
_wnl = nltk.WordNetLemmatizer()
from tqdm import tqdm
import scipy
from csv import DictReader
import pandas as pd
nltk.download('punkt')
nltk.download('wordnet')
# sklearn dependencies
from sklearn import feature_extraction
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import random
from collections import defaultdict


# Function for reading the dataset
class DataSet():
    def __init__(self, name="train", path="/content/drive/My Drive/641"):
        self.path = path

        print("Reading dataset")
        bodies = name+"_bodies.csv"
        stances = name+"_stances.csv"
        
        self.stances = self.read(stances)
       
        articles = self.read(bodies)
       
        self.articles = dict()

        #make the body ID an integer value
        for s in self.stances:
            s['Body ID'] = int(s['Body ID'])
        
        #copy all bodies into a dictionary
        for article in articles:
            self.articles[int(article['Body ID'])] = article['articleBody']

        print("Total stances: " + str(len(self.stances)))
        print("Total bodies: " + str(len(self.articles)))

    

    def read(self,filename):
        rows = []
        with open(self.path + "/" + filename, "r", encoding='utf-8') as table:
            r = DictReader(table)

            for line in r:
                rows.append(line)
        return rows


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Functions to form various features 

In [ ]:

def normalize_word(w):
    return _wnl.lemmatize(w).lower()


def get_tokenized_lemmas(s):
    return [normalize_word(t) for t in nltk.word_tokenize(s)]


def clean(s):
    # Cleans a string: Lowercasing, trimming, removing non-alphanumeric

    return " ".join(re.findall(r'\w+', s, flags=re.UNICODE)).lower()


def remove_stopwords(l):
    # Removes stopwords from a list of tokens
    return [w for w in l if w not in feature_extraction.text.ENGLISH_STOP_WORDS]


def gen_or_load_feats(feat_fn, headlines, bodies, feature_file):
    if not os.path.isfile(feature_file):
        feats = feat_fn(headlines, bodies)
        np.save(feature_file, feats)

    return np.load(feature_file)


def word_overlap_features(headlines, bodies):
    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        clean_headline = clean(headline)
        clean_body = clean(body)
        clean_headline = get_tokenized_lemmas(clean_headline)
        clean_body = get_tokenized_lemmas(clean_body)
        features = [
            len(set(clean_headline).intersection(clean_body)) / float(len(set(clean_headline).union(clean_body)))]
        X.append(features)
    return X


def refuting_features(headlines, bodies):
    _refuting_words = [
        'fake',
        'fraud',
        'hoax',
        'false',
        'deny', 'denies',
        # 'refute',
        'not',
        'despite',
        'nope',
        'doubt', 'doubts',
        'bogus',
        'debunk',
        'pranks',
        'retract'
    ]
    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        clean_headline = clean(headline)
        clean_headline = get_tokenized_lemmas(clean_headline)
        features = [1 if word in clean_headline else 0 for word in _refuting_words]
        X.append(features)
    return X


def polarity_features(headlines, bodies):
    _refuting_words = [
        'fake',
        'fraud',
        'hoax',
        'false',
        'deny', 'denies',
        'not',
        'despite',
        'nope',
        'doubt', 'doubts',
        'bogus',
        'debunk',
        'pranks',
        'retract'
    ]

    def calculate_polarity(text):
        tokens = get_tokenized_lemmas(text)
        return sum([t in _refuting_words for t in tokens]) % 2
    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        clean_headline = clean(headline)
        clean_body = clean(body)
        features = []
        features.append(calculate_polarity(clean_headline))
        features.append(calculate_polarity(clean_body))
        X.append(features)
    return np.array(X)


def ngrams(input, n):
    input = input.split(' ')
    output = []
    for i in range(len(input) - n + 1):
        output.append(input[i:i + n])
    return output


def chargrams(input, n):
    output = []
    for i in range(len(input) - n + 1):
        output.append(input[i:i + n])
    return output


def append_chargrams(features, text_headline, text_body, size):
    grams = [' '.join(x) for x in chargrams(" ".join(remove_stopwords(text_headline.split())), size)]
    grams_hits = 0
    grams_early_hits = 0
    grams_first_hits = 0
    for gram in grams:
        if gram in text_body:
            grams_hits += 1
        if gram in text_body[:255]:
            grams_early_hits += 1
        if gram in text_body[:100]:
            grams_first_hits += 1
    features.append(grams_hits)
    features.append(grams_early_hits)
    features.append(grams_first_hits)
    return features


def append_ngrams(features, text_headline, text_body, size):
    grams = [' '.join(x) for x in ngrams(text_headline, size)]
    grams_hits = 0
    grams_early_hits = 0
    for gram in grams:
        if gram in text_body:
            grams_hits += 1
        if gram in text_body[:255]:
            grams_early_hits += 1
    features.append(grams_hits)
    features.append(grams_early_hits)
    return features

def cosine_similarity_h(headlines, bodies):
	vectorizer = TfidfVectorizer(ngram_range=(1,2), lowercase=True, stop_words='english')#, max_features=1024)

	cos_sim_features = []
	for i in range(0, len(bodies)):
		body_headline = []
		body_headline.append(bodies[i])
		body_headline.append(headlines[i])
		tfidf = vectorizer.fit_transform(body_headline)

		cosine_similarity = (tfidf * tfidf.T).A
		cos_sim_features.append(cosine_similarity[0][1])

	
	cos_sim_array = np.array(cos_sim_features) 

	return cos_sim_array

def hand_features(headlines, bodies):

    def binary_co_occurence(headline, body):
        # Count how many times a token in the title
        # appears in the body text.
        bin_count = 0
        bin_count_early = 0
        for headline_token in clean(headline).split(" "):
            if headline_token in clean(body):
                bin_count += 1
            if headline_token in clean(body)[:255]:
                bin_count_early += 1
        return [bin_count, bin_count_early]

    def binary_co_occurence_stops(headline, body):
        # Count how many times a token in the title
        # appears in the body text. Stopwords in the title
        # are ignored.
        bin_count = 0
        bin_count_early = 0
        for headline_token in remove_stopwords(clean(headline).split(" ")):
            if headline_token in clean(body):
                bin_count += 1
                bin_count_early += 1
        return [bin_count, bin_count_early]

    def count_grams(headline, body):
        # Count how many times an n-gram of the title
        # appears in the entire body, and intro paragraph

        clean_body = clean(body)
        clean_headline = clean(headline)
        features = []
        features = append_chargrams(features, clean_headline, clean_body, 2)
        features = append_chargrams(features, clean_headline, clean_body, 8)
        features = append_chargrams(features, clean_headline, clean_body, 4)
        features = append_chargrams(features, clean_headline, clean_body, 16)
        features = append_ngrams(features, clean_headline, clean_body, 2)
        features = append_ngrams(features, clean_headline, clean_body, 3)
        features = append_ngrams(features, clean_headline, clean_body, 4)
        features = append_ngrams(features, clean_headline, clean_body, 5)
        features = append_ngrams(features, clean_headline, clean_body, 6)
        return features

    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        X.append(binary_co_occurence(headline, body)
                 + binary_co_occurence_stops(headline, body)
                 + count_grams(headline, body))


    return X


### Functions to generate splits

In [ ]:



def generate_hold_out_split (dataset, training = 0.8, base_dir="/content/drive/My Drive/641"):
    r = random.Random()
    r.seed(1489215)

    article_ids = list(dataset.articles.keys())  # get a list of article ids
    r.shuffle(article_ids)  # and shuffle that list


    training_ids = article_ids[:int(training * len(article_ids))]
    hold_out_ids = article_ids[int(training * len(article_ids)):]

    # write the split body ids out to files for future use
    with open(base_dir+ "/"+ "training_ids.txt", "w+") as f:
        f.write("\n".join([str(id) for id in training_ids]))

    with open(base_dir+ "/"+ "hold_out_ids.txt", "w+") as f:
        f.write("\n".join([str(id) for id in hold_out_ids]))



def read_ids(file,base):
    ids = []
    with open(base+"/"+file,"r") as f:
        for line in f:
           ids.append(int(line))
        return ids


def kfold_split(dataset, training = 0.8, n_folds = 10, base_dir="/content/drive/My Drive/641"):
    if not (os.path.exists(base_dir+ "/"+ "training_ids.txt")
            and os.path.exists(base_dir+ "/"+ "hold_out_ids.txt")):
        generate_hold_out_split(dataset,training,base_dir)

    training_ids = read_ids("training_ids.txt", base_dir)
    hold_out_ids = read_ids("hold_out_ids.txt", base_dir)

    folds = []
    for k in range(n_folds):
        folds.append(training_ids[int(k*len(training_ids)/n_folds):int((k+1)*len(training_ids)/n_folds)])

    return folds,hold_out_ids


def get_stances_for_folds(dataset,folds,hold_out):
    stances_folds = defaultdict(list)
    stances_hold_out = []
    for stance in dataset.stances:
        if stance['Body ID'] in hold_out:
            stances_hold_out.append(stance)
        else:
            fold_id = 0
            for fold in folds:
                if stance['Body ID'] in fold:
                    stances_folds[fold_id].append(stance)
                fold_id += 1

    return stances_folds,stances_hold_out


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

### Functions to generate the score

In [ ]:

LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
LABELS_RELATED = ['unrelated','related']
RELATED = LABELS[0:3]

def score_submission(gold_labels, test_labels):
    score = 0.0
    cm = [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]

    for i, (g, t) in enumerate(zip(gold_labels, test_labels)):
        g_stance, t_stance = g, t
        if g_stance == t_stance:
            score += 0.25
            if g_stance != 'unrelated':
                score += 0.50
        if g_stance in RELATED and t_stance in RELATED:
            score += 0.25

        cm[LABELS.index(g_stance)][LABELS.index(t_stance)] += 1

    return score, cm


def print_confusion_matrix(cm):
    lines = []
    header = "|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format('', *LABELS)
    line_len = len(header)
    lines.append("-"*line_len)
    lines.append(header)
    lines.append("-"*line_len)

    hit = 0
    total = 0
    for i, row in enumerate(cm):
        hit += row[i]
        total += sum(row)
        lines.append("|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format(LABELS[i],
                                                                   *row))
        lines.append("-"*line_len)
    print('\n'.join(lines))


def report_score(actual,predicted):
    score,cm = score_submission(actual,predicted)
    best_score, _ = score_submission(actual,actual)

    print_confusion_matrix(cm)
    print("Score: " +str(score) + " out of " + str(best_score) + "\t("+str(score*100/best_score) + "%)")
    return score*100/best_score




### Logistic Regression

In [ ]:


def generate_features(stances,dataset,name):
    h, b, y = [],[],[]

    for stance in stances:
        y.append(LABELS.index(stance['Stance']))
        h.append(stance['Headline'])
        b.append(dataset.articles[stance['Body ID']])

    X_overlap = gen_or_load_feats(word_overlap_features, h, b, "overlap."+name+".npy")
    X_refuting = gen_or_load_feats(refuting_features, h, b, "refuting."+name+".npy")
    X_polarity = gen_or_load_feats(polarity_features, h, b, "polarity."+name+".npy")
    X_hand = gen_or_load_feats(hand_features, h, b, "hand."+name+".npy")
    X_cosine=gen_or_load_feats(cosine_similarity_h,h,b,"cosine."+name+".npy")

    X = np.c_[X_hand, X_polarity, X_refuting, X_overlap,X_cosine]
    X = preprocessing.scale(X)
    return X,y
    



    
#Load the training dataset and generate folds
d = DataSet()
folds,hold_out = kfold_split(d,n_folds=10)
fold_stances, hold_out_stances = get_stances_for_folds(d,folds,hold_out)

# Load the competition dataset
competition_dataset = DataSet("competition_test")
X_competition, y_competition = generate_features(competition_dataset.stances, competition_dataset, "competition")
    
  
Xs = dict()
ys = dict()

# Load/Precompute all features now
X_holdout,y_holdout = generate_features(hold_out_stances,d,"holdout")
for fold in fold_stances:
    Xs[fold],ys[fold] = generate_features(fold_stances[fold],d,str(fold))


best_score = 0
best_fold = None

    



Reading dataset
Total stances: 49972
Total bodies: 1683
Reading dataset


0it [00:00, ?it/s]

Total stances: 25413
Total bodies: 904


25413it [01:24, 299.67it/s]
25413it [00:04, 5344.15it/s]
25413it [01:24, 301.24it/s]
25413it [01:30, 280.53it/s]
9622it [00:33, 288.70it/s]
9622it [00:01, 5275.61it/s]
9622it [00:33, 285.50it/s]
9622it [00:36, 261.38it/s]
4124it [00:14, 284.22it/s]
4124it [00:00, 5144.48it/s]
4124it [00:14, 279.54it/s]
4124it [00:15, 261.74it/s]
4663it [00:16, 278.65it/s]
4663it [00:00, 5299.73it/s]
4663it [00:16, 275.72it/s]
4663it [00:18, 253.35it/s]
3783it [00:11, 330.16it/s]
3783it [00:00, 5322.30it/s]
3783it [00:11, 324.67it/s]
3783it [00:12, 299.10it/s]
3388it [00:12, 276.14it/s]
3388it [00:00, 5363.76it/s]
3388it [00:12, 267.18it/s]
3388it [00:13, 252.46it/s]
3644it [00:12, 283.61it/s]
3644it [00:00, 5263.37it/s]
3644it [00:12, 280.54it/s]
3644it [00:13, 261.99it/s]
4644it [00:16, 288.89it/s]
4644it [00:00, 5453.75it/s]
4644it [00:16, 286.80it/s]
4644it [00:17, 265.95it/s]
3848it [00:14, 273.02it/s]
3848it [00:00, 5203.63it/s]
3848it [00:14, 268.80it/s]
3848it [00:15, 252.17it/s]
4273it [00:13, 

Score for fold 6 was - 0.7856937661591497


...

Score for fold 4 was - 0.8092717126913721


[Parallel(n_jobs=6)]: Done 150 out of 150 | elapsed:    0.2s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.1s


-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    93     |     0     |    614    |    55     |
-------------------------------------------------------------
| disagree  |    11     |     0     |    139    |    12     |
-------------------------------------------------------------
|  discuss  |    48     |     0     |   1621    |    131    |
-------------------------------------------------------------
| unrelated |     1     |     0     |    65     |   6832    |
-------------------------------------------------------------
Score: 3625.0 out of 4448.5	(81.4881420703608%)




[Parallel(n_jobs=6)]: Done 150 out of 150 | elapsed:    0.4s finished


-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    97     |     0     |   1598    |    208    |
-------------------------------------------------------------
| disagree  |    18     |     0     |    487    |    192    |
-------------------------------------------------------------
|  discuss  |    108    |     0     |   3833    |    523    |
-------------------------------------------------------------
| unrelated |     4     |     0     |    265    |   18080   |
-------------------------------------------------------------
Score: 9002.75 out of 11651.25	(77.26853341916103%)
______________________________________________________________________________________________________________________
                                            Headline  Body ID     Stance
0  Ferguson riots: Pregnant woman loses eye after...     2008  unrelated
1

In [ ]:
# Classifier for each fold
for fold in fold_stances:
    ids = list(range(len(folds)))
    del ids[fold]

    X_train = np.vstack(tuple([Xs[i] for i in ids]))
    y_train = np.hstack(tuple([ys[i] for i in ids]))

    X_test = Xs[fold]
    y_test = ys[fold]
        
    #tuned to best parameters     
    logisticregression= LogisticRegression(C = 1.0, solver="lbfgs", max_iter=4000,verbose=True)
    logisticregression.fit(X_train, y_train)
        
          
    predicted = [LABELS[int(a)] for a in logisticregression.predict(X_test)]   
       
        
    actual = [LABELS[int(a)] for a in y_test]

    fold_score, _ = score_submission(actual, predicted)
    max_fold_score, _ = score_submission(actual, actual)

    score = fold_score/max_fold_score

    print("Score for fold "+ str(fold) + " was - " + str(score))
    if score > best_score:
        best_score = score
        best_fold = logisticregression



#Run on Holdout set and report the final score on the holdout set
predicted = [LABELS[int(a)] for a in best_fold.predict(X_holdout)]
    
    
actual = [LABELS[int(a)] for a in y_holdout]

    
report_score(actual,predicted)
print("")
print("")

#Run on competition dataset
   
predicted = [LABELS[int(a)] for a in best_fold.predict(X_competition)]
actual = [LABELS[int(a)] for a in y_competition]
    
    
a=report_score(actual,predicted)
print("______________________________________________________________________________________________________________________")
# print(a)


    
# print(predicted)

body_id = []
headlines = []
stances = np.copy(np.array(predicted))

for i in range(len(competition_dataset.stances)):
    body_id.append(competition_dataset.stances[i]['Body ID'])
    headlines.append(competition_dataset.stances[i]['Headline'])
test_data = pd.DataFrame(columns=['Headline', 'Body ID', 'Stance'])
test_data['Headline'] = headlines
test_data['Body ID'] = body_id
test_data['Stance'] = stances
print(test_data.head())

test_data.to_csv('logistic_regression.csv', index=False, encoding='utf-8')


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Score for fold 6 was - 0.7861246768170066


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Score for fold 0 was - 0.817847511645011


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

Score for fold 4 was - 0.8044069251681213


[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.4s
[Parallel(n_jobs=6)]: Done 400 out of 400 | elapsed:    0.7s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.1s


-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    91     |     0     |    617    |    54     |
-------------------------------------------------------------
| disagree  |    11     |     0     |    138    |    13     |
-------------------------------------------------------------
|  discuss  |    42     |     0     |   1631    |    127    |
-------------------------------------------------------------
| unrelated |     1     |     0     |    61     |   6836    |
-------------------------------------------------------------
Score: 3633.0 out of 4448.5	(81.66797797010229%)




[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.6s
[Parallel(n_jobs=6)]: Done 400 out of 400 | elapsed:    1.1s finished


-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    111    |     0     |   1579    |    213    |
-------------------------------------------------------------
| disagree  |    17     |     0     |    485    |    195    |
-------------------------------------------------------------
|  discuss  |    129    |     0     |   3820    |    515    |
-------------------------------------------------------------
| unrelated |     3     |     0     |    254    |   18092   |
-------------------------------------------------------------
Score: 9006.5 out of 11651.25	(77.30071880699495%)
______________________________________________________________________________________________________________________
                                            Headline  Body ID     Stance
0  Ferguson riots: Pregnant woman loses eye after...     2008  unrelated
1 

In [ ]:
comp_test=pd.read_csv('/content/drive/My Drive/641/competition_test_stances.csv')

In [ ]:
comp_test

,Headline,Body ID,Stance
0,Ferguson riots: Pregnant woman loses eye after...,2008,unrelated
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,1550,unrelated
2,A Russian Guy Says His Justin Bieber Ringtone ...,2,unrelated
3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",1793,unrelated
4,Argentina's President Adopts Boy to End Werewo...,37,unrelated
...,...,...,...
25408,The success of the Affordable Care Act is a hu...,2582,agree
25409,The success of the Affordable Care Act is a hu...,2583,discuss
25410,The success of the Affordable Care Act is a hu...,2584,disagree
25411,The success of the Affordable Care Act is a hu...,2585,disagree


In [ ]:
#Calculating f1 score of individual stances type(Competition set)
from sklearn.metrics import f1_score

def calculate_f1scores(y_true, y_predicted):
    
    f1_macro = f1_score(y_true, y_predicted, average='macro')
    f1_classwise = f1_score(y_true, y_predicted, average=None, labels=["agree", "disagree", "discuss", "unrelated"])

    result = "F1 macro: {:.3f}".format(f1_macro * 100) + "% \n"
    result += "F1 agree: {:.3f}".format(f1_classwise[0] * 100) + "% \n"
    result += "F1 disagree: {:.3f}".format(f1_classwise[1] * 100) + "% \n"
    result += "F1 discuss: {:.3f}".format(f1_classwise[2] * 100) + "% \n"
    result += "F1 unrelated: {:.3f}".format(f1_classwise[3] * 100) + "% \n"
    return result
print(calculate_f1scores(predicted,actual))

F1 macro: 44.792% 
F1 agree: 10.264% 
F1 disagree: 0.000% 
F1 discuss: 72.062% 
F1 unrelated: 96.842% 



In [ ]:
from sklearn.metrics import classification_report

eval_report = classification_report(predicted,actual)
print('Test report', eval_report)

Test report               precision    recall  f1-score   support

       agree       0.06      0.43      0.10       260
    disagree       0.00      0.00      0.00         0
     discuss       0.86      0.62      0.72      6138
   unrelated       0.99      0.95      0.97     19015

    accuracy                           0.87     25413
   macro avg       0.48      0.50      0.45     25413
weighted avg       0.95      0.87      0.90     25413



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#General Accuracy
from sklearn.metrics import accuracy_score
accuracy_score(comp_test['Stance'],predicted)

0.8666037067642545